In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
plt.style.use('ggplot')

# Data preprocessing

__Change this vvvvvvv__

In [ ]:
CLUSTER = "WEBSTER"
model = 'TGCN'

## Merge models trained on other censoring strategies into the DataFrame

In [ ]:
import os
def add_censoring_strategy_predictions(df=None, censoring_level=1, censoring_strategy="Static", keep_obs=False, prefix='best_sweep'):
    directory_path = f"../predictions/{prefix}_{model.lower()}_{censoring_strategy.lower()}_{censoring_level}"
    if not os.path.exists(directory_path):
        raise NotADirectoryError(directory_path)

    preds_path = glob(f"{directory_path}/predictions_{model}_{CLUSTER}*.csv")[0] 
    df_preds = pd.read_csv(preds_path, parse_dates=["Date"], index_col=0)

    # Because we HAD a bug of double predictions being generated
    df_preds.dropna(inplace=True)

    # Assume we have forecast horizon of 1. Then we rename the columns to something more readable
    df_preds.rename(columns={
        CLUSTER+"_1": f"Censored Observations {censoring_strategy} {censoring_level}",
        CLUSTER+"_1_pred": f"Predicted {censoring_strategy} {censoring_level}",
        f"{CLUSTER}_1_true": f"True Observations"
    }, inplace=True)

    if not keep_obs:
        df_preds = df_preds.drop(f"True Observations", axis=1)

    if df is None:
        return df_preds
    return df.merge(df_preds, on="Date")

In [ ]:
df = add_censoring_strategy_predictions(None, 2, "Dynamic", keep_obs=True)
df = add_censoring_strategy_predictions(df, 1, "Dynamic")
df = add_censoring_strategy_predictions(df, 2, "Static")
df = add_censoring_strategy_predictions(df, 3, "Static")

df.head()

## Merge cluster capacity data onto the DataFrame

In [ ]:
plug_capacity = pd.read_csv("../data/MaximumPlugsPerCluster.csv", index_col=0, parse_dates=["StartDate", "EndDate"])
plug_capacity = plug_capacity[['StartDate', 'EndDate', 'Cluster', 'No. Plugs per Hub']]
plug_capacity = plug_capacity[plug_capacity.Cluster != 'SHERMAN']
plug_capacity = plug_capacity[plug_capacity.Cluster == CLUSTER]
plug_capacity.head()

In [ ]:
# For each cluster, set the last occurence's EndDate to be the last date in the dataset
clusters = plug_capacity['Cluster'].unique()
last_date = df.reset_index()['Date'].max()

# Update the last occurrence's EndDate for each cluster
for cluster in clusters:
    last_occurrence_index = plug_capacity[plug_capacity['Cluster'] == cluster].index[-1]
    plug_capacity.loc[last_occurrence_index, 'EndDate'] = last_date
plug_capacity.head()

In [ ]:
# Create a new DataFrame with a daily date range
start_date = plug_capacity['StartDate'].min()
end_date = plug_capacity['EndDate'].max()
daily_dates = pd.date_range(start_date, end_date, freq='D')
daily_df = pd.DataFrame({'Date': daily_dates})

# Merge the original DataFrame with the new DataFrame
merged_df = pd.merge_asof(daily_df, plug_capacity, left_on='Date', right_on='StartDate', direction='forward')

# Forward fill the missing values in the 'Cluster' and 'No. Plugs per Hub' columns
merged_df[['Cluster', 'No. Plugs per Hub']] = merged_df[['Cluster', 'No. Plugs per Hub']].ffill()

# Fill the remaining NaN values in the 'No. Plugs per Hub' column with the first available value (backwards fill)
merged_df['No. Plugs per Hub'] = merged_df['No. Plugs per Hub'].bfill()

# Drop unnecessary columns
result_df = merged_df.drop(['StartDate', 'EndDate'], axis=1)
result_df = result_df[result_df.Cluster == CLUSTER]
result_df

Merge cluster capacities onto the prediction dataframe

In [ ]:
df = pd.merge_asof(df, result_df, on="Date", direction="nearest")
df.head(-10)

In [ ]:
get_relevant_cols = lambda censoring_level, censoring_strategy: [
        'Date', 'No. Plugs per Hub', "True Observations",
        f"Censored Observations {censoring_strategy} {censoring_level}", f"Predicted {censoring_strategy} {censoring_level}"]

def plot_predictions(df, censoring_level, censoring_strategy):
    # Keep only relevant columns
    df_plot = df.copy()[get_relevant_cols(censoring_level, censoring_strategy)]
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.axhline(y=df_plot[df_plot.Date < '2019-05-30'][f'Censored Observations {censoring_strategy} {censoring_level}'].max(), linestyle='--', label='Censored Threshold', color='black', )
    df_plot[df_plot.Date < '2019-05-30']\
        .drop('No. Plugs per Hub', axis=1)\
        .drop(f"Censored Observations {censoring_strategy} {censoring_level}", axis=1)\
        .plot(x="Date", ax=ax, title=f"{model} Predictions for {CLUSTER} cluster with censoring strategy {censoring_strategy} {censoring_level}", ylabel='Number of sessions', colormap='tab10')

    # Set horizontal line at the maximum number of plugs
    ax.legend()
    fig.savefig(f"../Figures/prediction_timeseries_{model}_{CLUSTER}_{censoring_strategy}_{censoring_level}.png")
    plt.show()
plot_predictions(df, 2, "Dynamic")

In [ ]:
def plot_daily_max_sessions(df, censoring_level, censoring_strategy):
    df_plot = df.copy()[get_relevant_cols(censoring_level, censoring_strategy)]
    # Plot the highest prediction for each day, together with the highest recorded simultaneous number of sessions in a time interval
    fig, ax = plt.subplots(figsize=(20, 7))
    ax.axhline(y=df_plot[df_plot.Date < '2019-05-30'][f'Censored Observations {censoring_strategy} {censoring_level}'].max(), linestyle='--', label='Censored Threshold', color='black', )
    df_plot\
        .drop('No. Plugs per Hub', axis=1) \
        .drop(f"Censored Observations {censoring_strategy} {censoring_level}", axis=1) \
        .groupby(df_plot["Date"].dt.date).max() \
        .plot(x="Date", ax=ax, title=f"Max number of sessions for each day in cluster {CLUSTER}", colormap='tab10')
    ax.set_title(f"Max number of sessions for each day in cluster {CLUSTER}")
    ax.set_ylabel("Number of sessions")
    plt.show()

plot_daily_max_sessions(df, 2, "Dynamic")

In [ ]:
df.head()

In [ ]:
def plot_utilisation_per_day(df, censoring_level, censoring_strategy):
    df_util_plot = df.copy()
    cols = get_relevant_cols(censoring_level, censoring_strategy)
    cols.remove('Date')
    cols.remove('No. Plugs per Hub')
    # We should provide tau from the model training
    for col in cols:
        df_util_plot['tau_'+col] = df_util_plot[col].max()

    sessions_per_day = df_util_plot.groupby(df_util_plot["Date"].dt.date).sum().copy()

    # Calculate utilisation
    for col in cols:
        sessions_per_day['utilisation_'+col] = sessions_per_day[col] / sessions_per_day['No. Plugs per Hub']
    # Plot
    # drop all tau columns
    sessions_per_day = sessions_per_day[sessions_per_day.columns[sessions_per_day.columns.str.startswith('utilisation_')]]
    # remove utilisation_ from the column names
    sessions_per_day.columns = sessions_per_day.columns.str.replace('utilisation_', '')
    fig, ax = plt.subplots(figsize=(15, 5))
    sessions_per_day.plot(ax=ax, colormap='tab10')

    ax.set_title(f"Utilisation per day in cluster {CLUSTER}")
    ax.set_ylabel("Utilisation")
    fig.savefig(f"../Figures/daily_utilisation_{model}_{CLUSTER}_{censoring_strategy}_{censoring_level}.png")

plot_utilisation_per_day(df, 2, "Dynamic")

In [ ]:
plot_utilisation_per_day(df, 2, "Static")

In [ ]:
def calculate_utility(df, max_tau, censoring_level, censoring_strategy):
    """ 
    Calulcate the utility for a given cluster, given predictions, censored and true values of usage, for taus from 1 - max_tau.
    """
    cols = get_relevant_cols(censoring_level, censoring_strategy)
    cols.remove('Date')
    cols.remove('No. Plugs per Hub')
    util_values = {level: [] for level in cols}
    # calculate the utility for predicted usage, true usage and censored usage, for each tau
    for level, util_list in util_values.items():
        for tau_hypothetical in range(1, max_tau + 1):
            df_util = df.copy()
            # clip the values to the hypothetical tau (we use values from df instead of df_util to avoid clipping the values multiple times)
            df_util[level] = df[level].clip(upper=tau_hypothetical)
            # add the tau_hypothetical as a column
            df_util[f'tau_{tau_hypothetical}'] = tau_hypothetical
            # Sum the half-hourly values to get the daily values
            tmp = df_util.groupby(df_util["Date"].dt.date)[[level, f'tau_{tau_hypothetical}']].sum()
            # calculate the utility per day
            tmp['utilisation'] = (tmp[level] / tmp[f'tau_{tau_hypothetical}'])
            # add the mean utility to the list
            util_list.append(tmp['utilisation'].mean())

    return util_values
utility_values = calculate_utility(df, 12, 2, "Dynamic")
pd.DataFrame(utility_values)

In [ ]:
def plot_utility(df, max_tau, censoring_level, censoring_strategy, ax=None):
    utility_values = calculate_utility(df, max_tau, censoring_level, censoring_strategy)
    df_utilisation = pd.DataFrame(utility_values)
    df_utilisation['Capacity'] = df_utilisation.index + 1
    capacity = df['No. Plugs per Hub'].max()
    if ax is None:
        fig, ax = plt.subplots(figsize=(15, 8), sharey=True)
        # Only plot true observations if nothing else is plotted yet
        # Plot observed demand
        line, =  ax.plot(
            df_utilisation[df_utilisation.Capacity <= capacity]['Capacity'],
            df_utilisation[df_utilisation.Capacity <= capacity]["True Observations"],
            linestyle='-', marker='o', label="Utilisation for true demand"
        )
        color = line.get_color()
        ax.plot(
            df_utilisation[df_utilisation.Capacity >= capacity]['Capacity'],
            df_utilisation[df_utilisation.Capacity >= capacity]["True Observations"],
            linestyle='dashed', marker='o', color=color, alpha=0.5
        )
        fig.suptitle(f"Predicted vs observed utilisation for cluster {CLUSTER} for different capacities")

    max_observed_by_model = 0
    if censoring_strategy == "Dynamic":
        max_observed_by_model = capacity - censoring_level
    else:
        max_observed_by_model = censoring_level

    # Plot predicted demand
    
    line, = ax.plot(
        df_utilisation[df_utilisation.Capacity <= max_observed_by_model]['Capacity'],
        df_utilisation[df_utilisation.Capacity <= max_observed_by_model][f"Predicted {censoring_strategy} {censoring_level}"],
        linestyle='-', marker='o', label=f"Utilisation for predicted demand {censoring_strategy} {censoring_level}"
    )
    color = line.get_color()
    ax.plot(
        df_utilisation[df_utilisation.Capacity >= max_observed_by_model]['Capacity'],
        df_utilisation[df_utilisation.Capacity >= max_observed_by_model][f"Predicted {censoring_strategy} {censoring_level}"],
        linestyle='dashed', marker='o', color=color, alpha=0.5
    )

    ax.set_xlabel("Capacity")
    ax.set_ylabel("Utilisation")
    ax.yaxis.set_tick_params(labelleft=True)
    ax.legend()
    return ax

In [ ]:
ax = plot_utility(df, 12, 1, "Dynamic")
ax = plot_utility(df, 12, 2, "Dynamic", ax=ax)
ax = plot_utility(df, 12, 2, "Static", ax=ax)
ax = plot_utility(df, 12, 3, "Static", ax=ax)


ax.axhline(y=0.2, linestyle='--', color='g', label="20% utilisation")
plt.legend()
plt.show()

### Utilization is key (PWC)
Once an EV-charging station is built, all costs are essentially fixed, so utilization is key to achieving efficiency. Still, even a quite low utilization is likely to result in lines at busy times. In practice, the stand-alone, fast-charger industry uses a 20% utilization as a rule of thumb. But if a charging station is utilized at more than this 20% threshold, the operator will likely look to expand capacity or, more likely, add another site nearby.

## 24 hours forecast lead

In [ ]:
df_48 = add_censoring_strategy_predictions(None, 2, "Dynamic", keep_obs=True, prefix="forecast_48")
df_48 = add_censoring_strategy_predictions(df_48, 1, "Dynamic", prefix="forecast_48")
df_48 = add_censoring_strategy_predictions(df_48, 2, "Static", prefix="forecast_48")
df_48 = add_censoring_strategy_predictions(df_48, 3, "Static", prefix="forecast_48")

df_48 = pd.merge_asof(df_48, result_df, on="Date", direction="nearest")

df_48.head()

In [ ]:
plot_daily_max_sessions(df_48, 2, "Dynamic")

In [ ]:
utility_values = calculate_utility(df_48, 12, 2, "Dynamic")
ax = plot_utility(df_48, 12, 1, "Dynamic")
ax = plot_utility(df_48, 12, 2, "Dynamic", ax=ax)
ax = plot_utility(df_48, 12, 2, "Static", ax=ax)
ax = plot_utility(df_48, 12, 3, "Static", ax=ax)


ax.axhline(y=0.2, linestyle='--', color='g', label="20% utilisation")
plt.legend()
plt.show()

## Uncertainty plot 

In [ ]:
df_unc_i = add_censoring_strategy_predictions(None, 2, "Dynamic", keep_obs=True, prefix="uncertainty_1_forecast_1")
df_unc_i = add_censoring_strategy_predictions(df_unc_i, 2, "Dynamic", prefix="uncertainty_2_forecast_1")
df_unc_i = add_censoring_strategy_predictions(df_unc_i, 2, "Dynamic", prefix="uncertainty_3_forecast_1")
df_unc_i = add_censoring_strategy_predictions(df_unc_i, 2, "Dynamic", prefix="uncertainty_4_forecast_1")
df_unc_i.head()